# Tanzania country script

This script is formatted differently from other country scripts because Tanzania has useful data at both administrative level 1 and administrative level 2. To harmonize these, we need to aggregate the data from admin level 2 to admin level 1. But because there are admin boundary changes we first have to run the link function on the admin level 2 data, then aggregate it to admin level 1, then run the link function on admin level 1 data. This takes quite a while, longer than other scripts.

In [1]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import merge_admin1, agg_admin1, miss_pct, cleanPA, combine
from tools import save_hdf, save_npz, load_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import subprocess
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [2]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('../data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'EPSG:32735'
fn_shapes = sorted(glob.glob('../data/shapefile/fewsnet/TZ_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #


- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| 1982	 | TZ1982A1 | 24	| nan	| 0	| nan	| 0	|
| 1993	 | TZ1993A1 | 24	| TZ1993A2	| 121	| nan	| 0	|
| 1996	 | TZ1996A1 | 24	| TZ1996A2	| 122	| nan	| 0	|
| 1997	 | TZ1997A1 | 24	| TZ1997A2	| 125	| nan	| 0	|
| 2002	 | TZ2002A1 | 26	| TZ2002A2	| 129	| nan	| 0	|
| 2003	 | TZ2003A1 | 26	| TZ2003A2	| 130	| nan	| 0	|
| 2004	 | TZ2004A1 | 26	| TZ2004A2	| 130	| nan	| 0	|
| 2006	 | TZ2006A1 | 26	| TZ2006A2	| 133	| nan	| 0	|
| 2007	 | TZ2007A1 | 26	| TZ2007A2	| 136	| nan	| 0	|
| 2008	 | TZ2008A1 | 26	| TZ2008A2	| 143	| nan	| 0	|
| 2012	 | TZ2012A1 | 30	| TZ2012A2	| 169	| nan	| 0	|
----------------------------------------------- #


In [3]:
# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
#read in username and password
auth = tuple(json.loads(open('token.json', "r").read()))

parameters = {
    'format': 'json',
    'country': 'Tanzania',
   'product': ['R011','R012','R013','R014','R015','R017','R018'],
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionfacts/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())

df_origin = df.copy()
# ----------------------------------------------- #

In [ ]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting
# a.1) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# a.2) Early in the record the catch-all crop production system is 'all' rather than none. Make these to none-type
df.loc[df['crop_production_system'] == 'all', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# ----------------------------------------------- #
#Drop blank (nan) rows that are duplicates of rows that do have data
df = df[np.isfinite(df.value)]
df_admin1=df[df['admin_2'].isna()]
df = df.dropna(subset=['admin_2'])
# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df)
# remove a few stray generic "millet" crop names
df=df.loc[~(df['product']=='Millet')]
# crop list
Ocrop_list=list(df['product'].unique())
# rename crop list
crop_list = pd.Series(Ocrop_list).replace(product_category)
unique_list = list(set(crop_list))
# Initialize duplicate element position list
duplicate_positions = []
non_repeated_positions=[]
#
for element in unique_list:
    # identify positions
    positions = [i for i, x in enumerate(crop_list) if x == element]
    if len(positions) > 1:
        duplicate_positions.extend(positions)
    if len(positions) ==1:
        non_repeated_positions.extend(positions)
df=cleanPA(df,'admin_2','Tanzania','TZ',Ocrop_list, duplicate_positions,non_repeated_positions)
df['product'] = df['product'].replace(product_category)
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# # FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([TZ_Admin2_1993,TZ_Admin2_1996,TZ_Admin2_1997,TZ_Admin2_2002,TZ_Admin2_2003,TZ_Admin2_2004,TZ_Admin2_2006,TZ_Admin2_2007,TZ_Admin2_2008,TZ_Admin2_2012], axis=0)
PlotAdminShapes(shape_used, label=True)
# # # ----------------------------------------------- #



- Remove missing records ---------------------- #
Orignial data points: 45,535
Removed 151 "Missing Value" points
12,376/12,380 "Area Harvested" points are retained.
14,976/15,048 "Quantity Produced" points are retained.
14,940/15,014 "Yield" points are retained.
3,092/3,093 "Area Planted" points are retained.
Current data points: 45,384

- Minor changes are applied ------------------- #

- Basic information --------------------------- #
Data period: 1993 - 2010
29 grain types are found: Bambara groundnut, Banana (unspecified), Barley (Unspecified), Beans (mixed), Cassava, Chick Peas, Cocoyam, move to 1594aa, Cowpeas (Mixed), Field Peas, Gram (Green), Groundnuts (In Shell), Maize (Corn), Maize Grain (White), Millet, Millet (Bulrush), Millet (Finger), Mung bean, n.e.c., Pea (unspecified), Pigeon Peas, Potato (Irish), Rice (Paddy), Sesame Seed, Sorghum, Sorghum (Mixed), Soybean (unspecified), Sunflower seed, Sweet Potatoes, Wheat Grain, Yams
4 seasons are found: Annual (06-01), Long (05-

## Tanzania
Tanzania crop seasonal calendar

![FEWS NET](https://fews.net/sites/default/files/styles/large_width_880/public/2023-03/seasonal-calendar-tanzania.png?itok=locyPS3D)

![USDA](https://ipad.fas.usda.gov/rssiws/al/crop_calendar/images/eafrica_tz_calendar.png)


- comparison of boundaries
![image](./figures/TZ_admin_shapes.png)

- In Tanzannia, basd on the FEWS NET data, there were ten times of changes in administrative units: 1993, 1996, 1997,2002,2003,2004,2006,2007,2008 and 2021.

| year	 | Admin1   | # units   | Admin2   | # units   |
| :---: | :----:  | :----:   | :----:   | :---:  |
| 1982	 | TZ1982A1 | 24	| nan	| 0	|
| 1993	 | TZ1993A1 | 24	| TZ1993A2	| 121	|
| 1996	 | TZ1996A1 | 24	| TZ1996A2	| 122	|
| 1997	 | TZ1997A1 | 24	| TZ1997A2	| 125	|
| 2002	 | TZ2002A1 | 26	| TZ2002A2	| 129	|
| 2003	 | TZ2003A1 | 26	| TZ2003A2	| 130	|
| 2004	 | TZ2004A1 | 26	| TZ2004A2	| 130	|
| 2006	 | TZ2006A1 | 26	| TZ2006A2	| 133	|
| 2007	 | TZ2007A1 | 26	| TZ2007A2	| 136	|
| 2008	 | TZ2008A1 | 26	| TZ2008A2	| 143	|
| 2012	 | TZ2012A1 | 30	| TZ2012A2	| 169	|


- Here, we use **`ET2008A2`** to represent all admin-level 2 crop data 
- At admin-level 2,six crops have four seasons`Annual`,`Long` , `Short` and `Long`and`Short` 

In [ ]:
# # Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = TZ_Admin2_2008.copy().to_crs('epsg:32735')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #
# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# Link admin boundaries ------------------------- #
link_1993, over_1993 = FDW_PD_CreateAdminLink(TZ_Admin2_1993, TZ_Admin2_2008, 'ADMIN2', 'ADMIN2', prod, epsg)
link_1996, over_1996 = FDW_PD_CreateAdminLink(TZ_Admin2_1996, TZ_Admin2_2008, 'ADMIN2', 'ADMIN2', prod, epsg)
link_1997, over_1997 = FDW_PD_CreateAdminLink(TZ_Admin2_1997, TZ_Admin2_2008, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2002, over_2002 = FDW_PD_CreateAdminLink(TZ_Admin2_2002, TZ_Admin2_2008, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2003, over_2003 = FDW_PD_CreateAdminLink(TZ_Admin2_2003, TZ_Admin2_2008, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2004, over_2004 = FDW_PD_CreateAdminLink(TZ_Admin2_2004, TZ_Admin2_2008, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2006, over_2006 = FDW_PD_CreateAdminLink(TZ_Admin2_2006, TZ_Admin2_2008, 'ADMIN2', 'ADMIN2', prod, epsg)
link_2007, over_2007 = FDW_PD_CreateAdminLink(TZ_Admin2_2007, TZ_Admin2_2008, 'ADMIN2', 'ADMIN2', prod, epsg)

# Crop specific ratios
link_ratio_1993 = FDW_PD_RatioAdminLink(link_1993, prod, over_1993, mdx_pss)
link_ratio_1996 = FDW_PD_RatioAdminLink(link_1996, prod, over_1996, mdx_pss)
link_ratio_1997 = FDW_PD_RatioAdminLink(link_1997, prod, over_1997, mdx_pss)
link_ratio_2002 = FDW_PD_RatioAdminLink(link_2002, prod, over_2002, mdx_pss)
link_ratio_2003 = FDW_PD_RatioAdminLink(link_2003, prod, over_2003, mdx_pss)
link_ratio_2004 = FDW_PD_RatioAdminLink(link_2004, prod, over_2004, mdx_pss)
link_ratio_2006 = FDW_PD_RatioAdminLink(link_2006, prod, over_2006, mdx_pss)
link_ratio_2007 = FDW_PD_RatioAdminLink(link_2007, prod, over_2007, mdx_pss)

# Merge link_ratio
assert link_ratio_1993.keys() == link_ratio_2007.keys()
assert link_ratio_1996.keys() == link_ratio_2007.keys()
assert link_ratio_1997.keys() == link_ratio_2007.keys()
assert link_ratio_2002.keys() == link_ratio_2007.keys()
assert link_ratio_2003.keys() == link_ratio_2007.keys()
assert link_ratio_2004.keys() == link_ratio_2007.keys()
assert link_ratio_2006.keys() == link_ratio_2007.keys()

link_merged = [link_ratio_1993,link_ratio_1996, link_ratio_1997,link_ratio_2002,link_ratio_2003,link_ratio_2004,link_ratio_2006,link_ratio_2007]
fnids_new = list(link_merged[0].keys())
link_ratio = dict()
for fnid in fnids_new:
    container = []
    for link in link_merged:
        container.append(link[fnid])
    link_ratio[fnid] = pd.concat(container, axis=1)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=False)
# ----------------------------------------------- #

# Manual correction ----------------------------- #
crop_new = prod_new/area_new

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = (prod_new/area_new).T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "planting year"
cols = ['season_name','product','crop_production_system','planting_month','harvest_year','harvest_month','planting_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','planting_year','planting_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack= stack[names]
df = stack#[names]
df['indicator']=df['indicator'].replace({'area':'Area Harvested','production':'Quantity Produced','yield':'Yield'})

# aggregate up admin2 to admin1
country_iso, country_name = 'TZ', 'Tanzania'
df['fill_status'] = 'data'
df['missing_status']=''
country_code = df.country_code.unique()[0]
fnidYr = df.fnid.to_list()[0][2:6]

gdf = gpd.read_file('../data/shapefile/fewsnet/' + country_code + '_Admin1_' + fnidYr + '.shp')
fnidMap=gdf.set_index(['ADMIN1'])['FNID'].to_dict()

In [ ]:
#run the aggregation function to aggregate admin level 2 data up to admin level 1
aggYield1,aggActualAdmin1=agg_admin1(df,2,country_name,country_iso,fnidMap)


In [ ]:
#rename the admin 1 dataframe ahead of starting the whole process again at the admin 1 level
df = df_admin1

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df)
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# # FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat(
    [TZ_Admin1_1982, TZ_Admin1_1993, TZ_Admin1_1996, TZ_Admin1_1997, TZ_Admin1_2002, TZ_Admin1_2003, TZ_Admin1_2004,
     TZ_Admin1_2006, TZ_Admin1_2007, TZ_Admin1_2008, TZ_Admin1_2012], axis=0)
PlotAdminShapes(shape_used, label=True)
# # ----------------------------------------------- #


In [ ]:
# Define the latest shapefile ------------------- #
latest_level = 1
shape_latest = TZ_Admin1_2012.copy().to_crs('epsg:32735')
# ----------------------------------------------- #
df['product'] = df['product'].replace(product_category)
# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #

#Merge admin1 with amin2
df=pd.concat([df,aggActualAdmin1])

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0, 1]).unique()

In [ ]:
# Link admin boundaries ------------------------- #
link_1982, over_1982 = FDW_PD_CreateAdminLink(TZ_Admin1_1982, TZ_Admin1_2012, 'ADMIN1', 'ADMIN1', prod, epsg)
link_1993, over_1993 = FDW_PD_CreateAdminLink(TZ_Admin1_1993, TZ_Admin1_2012, 'ADMIN1', 'ADMIN1', prod, epsg)
link_1996, over_1996 = FDW_PD_CreateAdminLink(TZ_Admin1_1996, TZ_Admin1_2012, 'ADMIN1', 'ADMIN1', prod, epsg)
link_1997, over_1997 = FDW_PD_CreateAdminLink(TZ_Admin1_1997, TZ_Admin1_2012, 'ADMIN1', 'ADMIN1', prod, epsg)
link_2002, over_2002 = FDW_PD_CreateAdminLink(TZ_Admin1_2002, TZ_Admin1_2012, 'ADMIN1', 'ADMIN1', prod, epsg)
link_2003, over_2003 = FDW_PD_CreateAdminLink(TZ_Admin1_2003, TZ_Admin1_2012, 'ADMIN1', 'ADMIN1', prod, epsg)
link_2004, over_2004 = FDW_PD_CreateAdminLink(TZ_Admin1_2004, TZ_Admin1_2012, 'ADMIN1', 'ADMIN1', prod, epsg)
link_2006, over_2006 = FDW_PD_CreateAdminLink(TZ_Admin1_2006, TZ_Admin1_2012, 'ADMIN1', 'ADMIN1', prod, epsg)
link_2007, over_2007 = FDW_PD_CreateAdminLink(TZ_Admin1_2007, TZ_Admin1_2012, 'ADMIN1', 'ADMIN1', prod, epsg)
link_2008, over_2008 = FDW_PD_CreateAdminLink(TZ_Admin1_2008, TZ_Admin1_2012, 'ADMIN1', 'ADMIN1', prod, epsg)

# Crop specific ratios
link_ratio_1982 = FDW_PD_RatioAdminLink(link_1982, prod, over_1982, mdx_pss)
link_ratio_1993 = FDW_PD_RatioAdminLink(link_1993, prod, over_1993, mdx_pss)
link_ratio_1996 = FDW_PD_RatioAdminLink(link_1996, prod, over_1996, mdx_pss)
link_ratio_1997 = FDW_PD_RatioAdminLink(link_1997, prod, over_1997, mdx_pss)
link_ratio_2002 = FDW_PD_RatioAdminLink(link_2002, prod, over_2002, mdx_pss)
link_ratio_2003 = FDW_PD_RatioAdminLink(link_2003, prod, over_2003, mdx_pss)
link_ratio_2004 = FDW_PD_RatioAdminLink(link_2004, prod, over_2004, mdx_pss)
link_ratio_2006 = FDW_PD_RatioAdminLink(link_2006, prod, over_2006, mdx_pss)
link_ratio_2007 = FDW_PD_RatioAdminLink(link_2007, prod, over_2007, mdx_pss)
link_ratio_2008 = FDW_PD_RatioAdminLink(link_2008, prod, over_2008, mdx_pss)
# Merge link_ratio
assert link_ratio_1982.keys() == link_ratio_2008.keys()
assert link_ratio_1993.keys() == link_ratio_2008.keys()
assert link_ratio_1996.keys() == link_ratio_2008.keys()
assert link_ratio_1997.keys() == link_ratio_2008.keys()
assert link_ratio_2002.keys() == link_ratio_2008.keys()
assert link_ratio_2003.keys() == link_ratio_2008.keys()
assert link_ratio_2004.keys() == link_ratio_2008.keys()
assert link_ratio_2006.keys() == link_ratio_2008.keys()
assert link_ratio_2007.keys() == link_ratio_2008.keys()

link_merged = [link_ratio_1982, link_ratio_1993, link_ratio_1996, link_ratio_1997, link_ratio_2002, link_ratio_2003,
               link_ratio_2004, link_ratio_2006, link_ratio_2007, link_ratio_2008]
fnids_new = list(link_merged[0].keys())
link_ratio = dict()
for fnid in fnids_new:
    container = []
    for link in link_merged:
        container.append(link[fnid])
    link_ratio[fnid] = pd.concat(container, axis=1)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=False)
# ----------------------------------------------- #

In [ ]:
# Manual correction ----------------------------- #
crop_new = prod_new / area_new

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = (prod_new/area_new).T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "planting year"
cols = ['season_name','product','crop_production_system','planting_month','harvest_year','harvest_month','planting_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','planting_year','planting_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack = stack[names]
# ----------------------------------------------- #

# Final Processing ------------------------------ #
# Manual change of product name
product_category_custom = {

 'Banana (unspecified)':'Banana (unspecified)',
    'Bambara groundnut': 'Bambara groundnut',
    'Barley': 'Barley',
    'Beans (mixed)': 'Beans (mixed)',
    'Cassava': 'Cassava',
 'Chick Peas':'Chick Peas',
 'Cocoyam, move to 1594aa':'Cocoyam',
    'Cowpeas (Mixed)': 'Cowpeas',
 'Field Peas':'Field Peas',
 'Gram (Green)': 'Gram (Green)',
 'Groundnuts (In Shell)':'Groundnuts (In Shell)',
 'Maize':'Maize',
 'Millet':'Millet',
 'Mung bean, n.e.c.':'Mung bean',
 'Pea (unspecified)':'Pea (unspecified)',
 'Pigeon Peas':'Pigeon Peas',
 'Potato (Irish)':'Potato (Irish)',
 'Rice':'Rice',
 'Sesame Seed':'Sesame Seed',
 'Sorghum':'Sorghum',
 'Soybean (unspecified)':'Soybean',
 'Sugarcane (for sugar)':'Sugarcane',
 'Sunflower seed':'Sunflower seed',
 'Sweet Potatoes':'Sweet Potatoes',
 'Wheat':'Wheat',
 'Yams':'Yams'
}
assert set(stack['product'].unique()) == set(product_category_custom.keys())
stack['product'] = stack['product'].replace(product_category_custom)
for f, r in link_ratio.items():
    mdx = pd.MultiIndex.from_frame(r.index.to_frame().reset_index(drop=True).replace(product_category_custom))
    r.index = mdx
    link_ratio[f] = r
# Calibration of crop calendar using External Crop Calendar (ECC)
ecc = pd.read_csv('../data/crop_calendar/external_crop_calendar.csv')
ecc = ecc[ecc['country'] == 'Tanzania, United Republic of']
stack, link_ratio = FDW_PD_CaliSeasonYear(stack, ecc, link_ratio)
# None admin names
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# ----------------------------------------------- #

# Save data
fn_out = '../data/crop/adm_crop_production_TZ.csv'
stack.to_csv(fn_out); print(f'{fn_out} is saved.')
save_npz('../data/crop/adm_crop_production_TZ_ratio.npz', link_ratio)

In [ ]:
# Bar chart of national grain production
country_iso, country_name = 'TZ', 'Tanzania'
df = pd.read_csv('../data/crop/adm_crop_production_%s.csv' % country_iso, index_col=0)
df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_order = ['Maize', 'Wheat', 'Millet', 'Sorghum', 'Barley', 'Rice']
for season_name in ['Annual', 'Long', 'Short', 'Long/Dry']:
    footnote = 'National crop production in %s - %s' % (country_name, season_name)
    fn_save = '../figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote)#, fn_save)
    fig.show()

In [ ]:
# Lineplot of Production-Area-Yield (PAY) time-series
country_iso, country_name = 'TZ', 'Tanzania'
df = pd.read_csv('../data/crop/adm_crop_production_%s.csv' % country_iso, index_col=0)
df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [

    ['Maize', 'Annual'],
    ['Millet', 'Annual'],
    ['Sorghum', 'Annual'],
    ['Wheat', 'Annual'],
    ['Rice', 'Annual'],
    ['Barley','Annual']
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield (PAY) time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = '../figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    fig.show()


# Outputting these plots are no longer default, so this cell is markdown
## Calibrated PAY time-series per FNID
from tools_graphic import PlotLineCropTS

country_iso, country_name = 'TZ', 'Tanzania'
df = pd.read_hdf('../data/crop/adm_crop_production_%s.hdf' % country_iso)
df['year'] = df['harvest_year']
link_ratio = load_npz('../data/crop/adm_crop_production_%s_ratio.npz' % country_iso)
year_all = np.arange(df['year'].min(), df['year'].max() + 1)
product_season = [

    ['Maize', 'Annual'],
    ['Millet', 'Annual'],
    ['Sorghum', 'Annual'],
    ['Wheat', 'Annual'],
    ['Rice', 'Annual'],
]
for product_name, season_name in product_season:
    sub = df[
        (df['product'] == product_name) &
        (df['season_name'] == season_name)
        ]
    for fnid in sub['fnid'].unique():
        sub_fps = sub[sub['fnid'] == fnid]
        fn_save = '../figures/crop_calibrated/%s_%s_%s_%s.png' % (country_iso, product_name, season_name, fnid)
        fig = PlotLineCropTS(sub_fps, fnid, product_name, season_name, link_ratio, year_all, fn_save)
